# 000 preprocess
---

In [1]:
import os, gc, yaml, glob, pickle
import time
import random
from pathlib import Path
from types import SimpleNamespace

import numpy as np
import polars as pl
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append("../src/utils")
from metric import score
from data import sep, show_df, glob_walk, set_seed, save_config_yaml, dict_to_namespace

In [2]:
# ===============
# Config
# ===============
config = SimpleNamespace(
    INPUT_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw"),
    TRAIN_TRACKING_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw") / "train_tracking",
    TRAIN_ANNOTATION_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw") / "train_annotation",
    TEST_TRACKING_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw") / "test_tracking",
    seed = 42,
    n_splits = 5,
    INDEX_COLS = [
        "video_id",
        "agent_mouse_id",
        "target_mouse_id",
        "video_frame",
    ],
    BODY_PARTS = [
        "ear_left",
        "ear_right",
        "nose",
        "neck",
        "body_center",
        "lateral_left",
        "lateral_right",
        "hip_left",
        "hip_right",
        "tail_base",
        "tail_tip",
    ],
    SELF_BEHAVIORS = [
        "biteobject",
        "climb",
        "dig",
        "exploreobject",
        "freeze",
        "genitalgroom",
        "huddle",
        "rear",
        "rest",
        "run",
        "selfgroom",
    ],
    PAIR_BEHAVIORS = [
        "allogroom",
        "approach",
        "attack",
        "attemptmount",
        "avoid",
        "chase",
        "chaseattack",
        "defend",
        "disengage",
        "dominance",
        "dominancegroom",
        "dominancemount",
        "ejaculate",
        "escape",
        "flinch",
        "follow",
        "intromit",
        "mount",
        "reciprocalsniff",
        "shepherd",
        "sniff",
        "sniffbody",
        "sniffface",
        "sniffgenital",
        "submit",
        "tussle",
    ],
)

In [3]:
# ===============
# read data
# ===============
train_dataframe = pl.read_csv(config.INPUT_DIR / "train.csv")
sep("train_dataframe"); show_df(train_dataframe)

train_dataframe
(8789, 38)


lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,mouse2_sex,mouse2_id,mouse2_age,mouse2_condition,mouse3_strain,mouse3_color,mouse3_sex,mouse3_id,mouse3_age,mouse3_condition,mouse4_strain,mouse4_color,mouse4_sex,mouse4_id,mouse4_age,mouse4_condition,frames_per_second,video_duration_sec,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
str,i64,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,f64,f64,f64,i64,i64,f64,f64,str,str,str,str,str
"""AdaptableSnail""",44566106,"""CD-1 (ICR)""","""white""","""male""",10.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",24.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",38.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",51.0,"""8-12 weeks""","""wireless device""",30.0,615.6,16.0,1228,1068,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""
"""AdaptableSnail""",143861384,"""CD-1 (ICR)""","""white""","""male""",3.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",17.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",31.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",44.0,"""8-12 weeks""",null,25.0,3599.0,9.7,968,608,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""
"""AdaptableSnail""",209576908,"""CD-1 (ICR)""","""white""","""male""",7.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",21.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",35.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",48.0,"""8-12 weeks""",null,30.0,615.2,16.0,1266,1100,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""


In [4]:
# test
test_dataframe = pl.read_csv(config.INPUT_DIR / "test.csv")
sep("test_dataframe"); show_df(test_dataframe)

# sub
sub_dataframe = pl.read_csv(config.INPUT_DIR / "sample_submission.csv")
sep("sub_dataframe"); show_df(sub_dataframe)

test_dataframe
(1, 38)


lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,mouse2_sex,mouse2_id,mouse2_age,mouse2_condition,mouse3_strain,mouse3_color,mouse3_sex,mouse3_id,mouse3_age,mouse3_condition,mouse4_strain,mouse4_color,mouse4_sex,mouse4_id,mouse4_age,mouse4_condition,frames_per_second,video_duration_sec,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
str,i64,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,f64,f64,f64,i64,i64,f64,f64,str,str,str,str,str
"""AdaptableSnail""",438887472,"""CD-1 (ICR)""","""white""","""male""",13.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",27.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",41.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",53.0,"""8-12 weeks""","""wireless device""",30.0,614.7,16.0,1214,1090,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""


sub_dataframe
(1, 7)


row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
i64,i64,str,str,str,i64,i64
0,438887472,"""mouse1""","""mouse2""","""sniff""",0,1


In [5]:
train_dataframe["video_id"].unique().len() # 全てユニークな動画ID

8789

In [6]:
train_dataframe.filter(pl.col("behaviors_labeled").is_null()).shape
# 7941/8789行がラベルなし

(7941, 38)

In [7]:
train_dataframe.filter(pl.col("behaviors_labeled").is_not_null()).shape

(848, 38)

In [8]:
#　プロセス確認用
# (train_dataframe.filter(pl.col("behaviors_labeled").is_not_null())
# .select(
#     pl.col("lab_id"),
#     pl.col("video_id"),
#     pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
# )
#     .explode("behaviors_labeled_list")
# )

In [9]:
# =============================
# preprocess behavior labels
# =============================
train_behavior_dataframe = (
    train_dataframe.filter(pl.col("behaviors_labeled").is_not_null())
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        # list化(["mouse1,mouse2,approach", "mouse1,mouse2,attack", … "mouse4,self,rear"])
        pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
    )
    # listをexplodeして1行ずつに分解(848->4_919行)
    .explode("behaviors_labeled_list")
    .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled_element").str.split(",").list[0].str.replace_all("'", "").alias("agent"),
        pl.col("behaviors_labeled_element").str.split(",").list[1].str.replace_all("'", "").alias("target"),
        pl.col("behaviors_labeled_element").str.split(",").list[2].str.replace_all("'", "").alias("behavior"),
    )
)

train_self_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(config.SELF_BEHAVIORS))
train_pair_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(config.PAIR_BEHAVIORS))
sep("train_self_behavior_dataframe")
show_df(train_self_behavior_dataframe)
sep("train_pair_behavior_dataframe")
show_df(train_pair_behavior_dataframe)

train_self_behavior_dataframe
(594, 5)


lab_id,video_id,agent,target,behavior
str,i64,str,str,str
"""AdaptableSnail""",44566106,"""mouse1""","""self""","""rear"""
"""AdaptableSnail""",44566106,"""mouse2""","""self""","""rear"""
"""AdaptableSnail""",44566106,"""mouse3""","""self""","""rear"""


train_pair_behavior_dataframe
(4325, 5)


lab_id,video_id,agent,target,behavior
str,i64,str,str,str
"""AdaptableSnail""",44566106,"""mouse1""","""mouse2""","""approach"""
"""AdaptableSnail""",44566106,"""mouse1""","""mouse2""","""attack"""
"""AdaptableSnail""",44566106,"""mouse1""","""mouse2""","""avoid"""


In [10]:
import json

# ラベル付き動画だけ
train_labeled = (
    train_dataframe
    .filter(pl.col("behaviors_labeled").is_not_null())
    .select(
        "lab_id",
        "video_id",
        "body_parts_tracked",
        "behaviors_labeled",
        "frames_per_second",
        "video_duration_sec",
    )
)

# 動画メタ情報 (body_parts_tracked も list 化しておく)
video_meta = (
    train_labeled
    .with_columns([
        pl.col("body_parts_tracked")
          .map_elements(json.loads, return_dtype=pl.List(pl.Utf8))
          .alias("body_parts_tracked_list"),
        pl.col("behaviors_labeled")
          .map_elements(json.loads, return_dtype=pl.List(pl.Utf8))
          .alias("behaviors_labeled_list"),
    ])
)

# behaviors_labeled を縦持ちに
behavior_pairs_raw = (
    video_meta
    .select(
        "lab_id",
        "video_id",
        "behaviors_labeled_list",
    )
    .explode("behaviors_labeled_list")  # 1行 = 1 "mouse1,mouse2,attack" など
    .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
    # ここでまずクォートを全部落とす
    .with_columns(
        pl.col("behaviors_labeled_element")
          .str.replace_all("'", "")
          .str.replace_all('"', "")
          .alias("behaviors_clean")
    )
    .with_columns(
        pl.col("behaviors_clean").str.split(",").alias("parts")
    )
    .select(
        "lab_id",
        "video_id",
        pl.col("parts").list.get(0).alias("agent_str"),
        pl.col("parts").list.get(1).alias("target_str"),
        pl.col("parts").list.get(2).alias("action"),
    )
)

# Python 側で mouse_id を安全にパースする関数
def parse_mouse_id(s):
    if s is None:
        return None
    s = str(s).strip()
    # クォートとダブルクォートを除去
    s = s.replace("'", "").replace('"', "")
    if s.lower() == "self":  # target 用
        return None
    s = s.replace("mouse", "")
    # 数字だけ残す
    digits = "".join(ch for ch in s if ch.isdigit())
    return int(digits) if digits else None

behavior_pairs = (
    behavior_pairs_raw
    .with_columns([
        pl.col("agent_str")
          .map_elements(parse_mouse_id, return_dtype=pl.Int8)
          .alias("agent_mouse_id"),
        pl.col("target_str")
          .map_elements(parse_mouse_id, return_dtype=pl.Int8)
          .alias("target_mouse_raw_id"),
    ])
    # target が self の場合は agent をコピー
    .with_columns(
        pl.when(pl.col("target_mouse_raw_id").is_null())
          .then(pl.col("agent_mouse_id"))
          .otherwise(pl.col("target_mouse_raw_id"))
          .alias("target_mouse_id")
    )
    .drop("target_mouse_raw_id")
    .with_columns(
        pl.when(pl.col("action").is_in(config.SELF_BEHAVIORS))
          .then(pl.lit("self"))
          .otherwise(pl.lit("pair"))
          .alias("behavior_type")
    )
    .select(
        "lab_id",
        "video_id",
        "agent_mouse_id",
        "target_mouse_id",
        "action",
        "behavior_type",
    )
    .unique()
)

train_self_behavior_dataframe = behavior_pairs.filter(pl.col("behavior_type") == "self")
train_pair_behavior_dataframe = behavior_pairs.filter(pl.col("behavior_type") == "pair")

sep("train_self_behavior_dataframe"); show_df(train_self_behavior_dataframe, 10, True)
sep("train_pair_behavior_dataframe"); show_df(train_pair_behavior_dataframe, 10, True)

/tmp/ipykernel_1958556/3721545516.py:22: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("body_parts_tracked").map_elements(json.loads)
with this one instead:
  + pl.col("body_parts_tracked").str.json_decode()

  .map_elements(json.loads, return_dtype=pl.List(pl.Utf8))
/tmp/ipykernel_1958556/3721545516.py:25: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("behaviors_labeled").map_elements(json.loads)
with this one instead:
  + pl.col("behaviors_labeled").str.json_decode()

  .map_elements(json.loads, return_dtype=pl.List(pl.Utf8))


train_self_behavior_dataframe
(594, 6)


lab_id,video_id,agent_mouse_id,target_mouse_id,action,behavior_type
str,i64,i8,i8,str,str
"""GroovyShrew""",1864237378,1,1,"""selfgroom""","""self"""
"""GroovyShrew""",2114187055,1,1,"""dig""","""self"""
"""TranquilPanther""",675808983,2,2,"""selfgroom""","""self"""
"""NiftyGoldfinch""",101686631,1,1,"""exploreobject""","""self"""
"""InvincibleJellyfish""",1386249360,1,1,"""selfgroom""","""self"""
"""AdaptableSnail""",1260392287,3,3,"""rear""","""self"""
"""ElegantMink""",715350314,2,2,"""rear""","""self"""
"""GroovyShrew""",1205169652,2,2,"""run""","""self"""
"""UppityFerret""",86824777,2,2,"""rear""","""self"""


lab_id,video_id,agent_mouse_id,target_mouse_id,action,behavior_type
str,i64,i8,i8,str,str
"""GroovyShrew""",2101706406,1,1,"""dig""","""self"""
"""UppityFerret""",532567355,1,1,"""huddle""","""self"""
"""AdaptableSnail""",678426900,3,3,"""rear""","""self"""
"""GroovyShrew""",2114187055,2,2,"""dig""","""self"""
"""AdaptableSnail""",351967631,3,3,"""rear""","""self"""
"""NiftyGoldfinch""",425399867,1,1,"""climb""","""self"""
"""NiftyGoldfinch""",960582355,1,1,"""biteobject""","""self"""
"""ElegantMink""",1045757672,2,2,"""selfgroom""","""self"""
"""GroovyShrew""",1057262087,1,1,"""dig""","""self"""


train_pair_behavior_dataframe
(4270, 6)


lab_id,video_id,agent_mouse_id,target_mouse_id,action,behavior_type
str,i64,i8,i8,str,str
"""InvincibleJellyfish""",1205744881,1,2,"""dominancegroom""","""pair"""
"""TranquilPanther""",1735658729,1,2,"""sniff""","""pair"""
"""BoisterousParrot""",1459695188,2,1,"""shepherd""","""pair"""
"""GroovyShrew""",5119542,2,1,"""sniffgenital""","""pair"""
"""AdaptableSnail""",878123481,1,3,"""approach""","""pair"""
"""PleasantMeerkat""",843765839,2,1,"""chase""","""pair"""
"""PleasantMeerkat""",886803007,1,2,"""escape""","""pair"""
"""CalMS21_task1""",1860623663,1,2,"""approach""","""pair"""
"""PleasantMeerkat""",1177059156,1,2,"""escape""","""pair"""


lab_id,video_id,agent_mouse_id,target_mouse_id,action,behavior_type
str,i64,i8,i8,str,str
"""CalMS21_supplemental""",2030380897,1,2,"""sniff""","""pair"""
"""CalMS21_supplemental""",1132476818,1,2,"""mount""","""pair"""
"""AdaptableSnail""",351967631,1,3,"""avoid""","""pair"""
"""PleasantMeerkat""",372539857,2,1,"""escape""","""pair"""
"""JovialSwallow""",75620690,1,2,"""attack""","""pair"""
"""CautiousGiraffe""",1657896715,2,1,"""sniff""","""pair"""
"""ElegantMink""",464176860,1,2,"""sniff""","""pair"""
"""AdaptableSnail""",678426900,3,4,"""chaseattack""","""pair"""
"""CalMS21_supplemental""",1458368239,1,2,"""sniffgenital""","""pair"""


In [11]:
sep("behavior_pairs"); show_df(behavior_pairs, 10, True)

behavior_pairs
(4864, 6)


lab_id,video_id,agent_mouse_id,target_mouse_id,action,behavior_type
str,i64,i8,i8,str,str
"""InvincibleJellyfish""",1205744881,1,2,"""dominancegroom""","""pair"""
"""TranquilPanther""",1735658729,1,2,"""sniff""","""pair"""
"""BoisterousParrot""",1459695188,2,1,"""shepherd""","""pair"""
"""GroovyShrew""",5119542,2,1,"""sniffgenital""","""pair"""
"""AdaptableSnail""",878123481,1,3,"""approach""","""pair"""
"""GroovyShrew""",1864237378,1,1,"""selfgroom""","""self"""
"""PleasantMeerkat""",843765839,2,1,"""chase""","""pair"""
"""PleasantMeerkat""",886803007,1,2,"""escape""","""pair"""
"""CalMS21_task1""",1860623663,1,2,"""approach""","""pair"""


lab_id,video_id,agent_mouse_id,target_mouse_id,action,behavior_type
str,i64,i8,i8,str,str
"""PleasantMeerkat""",372539857,2,1,"""escape""","""pair"""
"""JovialSwallow""",75620690,1,2,"""attack""","""pair"""
"""ElegantMink""",1045757672,2,2,"""selfgroom""","""self"""
"""CautiousGiraffe""",1657896715,2,1,"""sniff""","""pair"""
"""ElegantMink""",464176860,1,2,"""sniff""","""pair"""
"""AdaptableSnail""",678426900,3,4,"""chaseattack""","""pair"""
"""GroovyShrew""",1057262087,1,1,"""dig""","""self"""
"""GroovyShrew""",1194163628,2,2,"""dig""","""self"""
"""CalMS21_supplemental""",1458368239,1,2,"""sniffgenital""","""pair"""


# Tracking data

In [12]:
def build_video_path_dict(paths: list[Path]) -> dict[int, Path]:
    video2path = {}
    for p in paths:
        # ファイル名 = "44566106.parquet" という前提
        vid = int(p.stem)
        video2path[vid] = p
    return video2path

In [13]:
# =============================
# gather tracking paths
# =============================
tracking_paths = glob_walk(config.TRAIN_TRACKING_DIR, "**/*.parquet")
video2tracking = build_video_path_dict(tracking_paths) # 全動画分のtrackingパスを辞書化
print(f"length of tracking paths: {len(tracking_paths)}")
print(tracking_paths[:3])

length of tracking paths: 8789
[PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_tracking/AdaptableSnail/1212811043.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_tracking/AdaptableSnail/1260392287.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_tracking/AdaptableSnail/1351098077.parquet')]


In [14]:
df_tracking_sample = pl.read_parquet(tracking_paths[2])
sep("df_tracking_sample"); show_df(df_tracking_sample)

df_tracking_sample
(1617776, 5)


video_frame,mouse_id,bodypart,x,y
i32,i8,str,f32,f32
0,1,"""body_center""",1149.209961,445.333008
0,1,"""ear_left""",1157.660034,516.973999
0,1,"""ear_right""",1116.16394,485.903015


In [15]:
print(*list(df_tracking_sample["video_frame"].unique())[:10])

0 1 2 3 4 5 6 7 8 9


In [16]:
df_pivot = (
    df_tracking_sample
    .pivot(
        index=["video_frame", "mouse_id"],
        on="bodypart",      
        values=["x", "y"],
    )
    .sort(["video_frame", "mouse_id"])
)
sep("df_pivot"); show_df(df_pivot, 10, True)

# 2_677_006 -> 318_646行に削減

df_pivot
(242707, 22)


video_frame,mouse_id,x_body_center,x_ear_left,x_ear_right,x_lateral_left,x_lateral_right,x_nose,x_tail_base,x_tail_tip,x_tail_midpoint,x_neck,y_body_center,y_ear_left,y_ear_right,y_lateral_left,y_lateral_right,y_nose,y_tail_base,y_tail_tip,y_tail_midpoint,y_neck
i32,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1,1149.209961,1157.660034,1116.16394,1172.079956,1132.894043,1116.237061,1165.696045,330.029999,null,null,445.333008,516.973999,485.903015,476.265015,398.407013,526.552002,389.093994,367.631989,null,null
0,2,1104.375977,1123.755981,1072.952026,1128.312988,1075.534058,1110.120972,1081.697998,980.187012,1030.911011,null,304.947998,332.627014,342.309998,290.011993,308.338989,380.158997,274.985992,188.085999,234.401993,null
0,4,264.707001,245.007996,237.647995,284.531006,261.359009,188.632996,295.811005,null,342.378998,null,529.129028,561.588013,517.10199,554.869019,495.196014,538.15802,503.34201,null,447.311005,null
1,1,1151.474976,1152.540039,1138.628052,1171.180054,1164.267944,1102.490967,1163.123047,330.640991,null,null,452.80899,521.947021,405.372986,482.645996,402.563995,383.053986,396.161011,367.600006,null,null
1,2,1105.51001,1124.618042,1072.762939,1134.557007,1082.232056,null,1094.462036,980.932007,1047.628052,null,315.009003,336.235992,340.915985,312.863007,302.312988,null,280.283997,194.697006,231.712006,null
1,4,269.755005,246.979004,241.117996,291.902008,264.782013,196.436996,303.700989,null,346.834991,null,527.312988,562.159973,521.320007,546.619995,492.790009,527.757019,502.001007,null,446.829987,null
2,1,1151.827026,1153.832031,1132.987061,1171.368042,1156.902954,null,1165.732056,330.735992,null,null,453.035004,520.281982,405.522003,482.233002,403.054993,null,397.067993,369.046997,null,null
2,2,1103.655029,1119.79895,1068.567017,1132.973022,1075.192993,1099.938965,1094.609985,981.106995,1048.917969,null,314.234985,335.475006,340.154999,316.86499,305.850006,385.057007,281.042999,194.479004,231.863007,null
2,4,273.295013,248.285995,246.292007,287.394989,271.584991,196.388,304.661987,null,348.813995,null,527.046997,558.210022,520.366028,548.804993,495.036987,517.812012,502.984985,null,439.544006,null


video_frame,mouse_id,x_body_center,x_ear_left,x_ear_right,x_lateral_left,x_lateral_right,x_nose,x_tail_base,x_tail_tip,x_tail_midpoint,x_neck,y_body_center,y_ear_left,y_ear_right,y_lateral_left,y_lateral_right,y_nose,y_tail_base,y_tail_tip,y_tail_midpoint,y_neck
i32,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
81776,4,635.982971,620.5,617.804016,null,null,646.705017,null,608.919983,null,null,101.098,110.539001,111.748001,null,null,156.102997,null,163.878998,null,null
81777,1,602.534973,null,null,620.192017,597.559998,null,574.695984,564.442993,557.835999,null,68.638,null,null,82.982002,89.660004,null,77.628998,108.317001,126.516998,null
81777,2,581.731995,617.210999,575.77301,614.906982,552.515991,622.234985,579.573975,null,594.070984,null,927.046997,938.16803,972.663025,910.075989,937.380981,999.974976,890.33197,null,817.927979,null
81777,4,635.952026,620.580017,617.700012,null,null,646.632019,null,609.286987,null,null,100.966003,110.718002,111.996002,null,null,155.968002,null,163.867004,null,null
81778,1,602.383972,null,null,620.070984,597.575989,null,574.690979,564.47699,557.830994,null,68.906998,null,null,83.053001,89.403,null,77.626999,108.266998,126.473,null
81778,2,581.960999,619.184021,575.815979,615.651978,550.783997,621.140991,579.622986,null,594.093994,null,927.250977,940.935974,972.817993,909.927979,930.218994,999.776978,890.445007,null,817.734985,null
81778,4,635.971985,620.562988,617.469971,null,null,646.69397,null,609.655029,null,null,101.106003,110.769997,112.200996,null,null,155.854996,null,163.785995,null,null
81779,1,602.387024,null,null,620.077026,597.637024,605.573975,574.737976,564.504028,557.848022,null,68.956001,null,null,83.086998,89.438004,106.528,77.634003,108.267998,126.732002,null
81779,2,581.28302,619.11499,575.786011,616.012024,551.484985,620.854004,579.580017,null,594.012024,null,926.465027,940.682983,972.708008,910.296997,931.075012,999.987976,890.598999,null,818.166016,null


In [17]:
def rename_agent(col: str) -> str:
    if col in ("video_frame", "mouse_id"):
        return col
    if col.startswith("x_"):
        return "x_agent_" + col[2:]   # "x_" の2文字を切って後ろだけ使う
    if col.startswith("y_"):
        return "y_agent_" + col[2:]
    return col

def rename_target(col: str) -> str:
    if col in ("video_frame", "mouse_id"):
        return col
    if col.startswith("x_"):
        return "x_target_" + col[2:]
    if col.startswith("y_"):
        return "y_target_" + col[2:]
    return col

In [18]:
def make_pair_features(df_pivot: pl.DataFrame, agent_id: int, target_id: int) -> pl.DataFrame:
    # agentの系列
    agent = (
        df_pivot
        .filter(pl.col("mouse_id") == agent_id)
        .rename(rename_agent)
    )
    # targetの系列
    target = (
        df_pivot
        .filter(pl.col("mouse_id") == target_id)
        .rename(rename_target)
    )
    # video_frame で結合
    pair = (
        agent.drop("mouse_id")
        .join(target.drop("mouse_id"), on="video_frame", how="inner")
        .sort("video_frame")
    )
    # show_df(pair, 10, True)
    # 相対位置・距離などを追加
    pair = pair.with_columns(
        (pl.col("x_target_body_center") - pl.col("x_agent_body_center")).alias("dx"),
        (pl.col("y_target_body_center") - pl.col("y_agent_body_center")).alias("dy"),
    ).with_columns(
        (pl.col("dx") ** 2 + pl.col("dy") ** 2).sqrt().alias("dist"),
    )
    return pair

In [19]:
df_tracking_sample_pair = make_pair_features(df_pivot, agent_id=1, target_id=2)
sep("df_tracking_sample_pair"); show_df(df_tracking_sample_pair, 10, True)

df_tracking_sample_pair
(80062, 44)


video_frame,x_agent_body_center,x_agent_ear_left,x_agent_ear_right,x_agent_lateral_left,x_agent_lateral_right,x_agent_nose,x_agent_tail_base,x_agent_tail_tip,x_agent_tail_midpoint,x_agent_neck,y_agent_body_center,y_agent_ear_left,y_agent_ear_right,y_agent_lateral_left,y_agent_lateral_right,y_agent_nose,y_agent_tail_base,y_agent_tail_tip,y_agent_tail_midpoint,y_agent_neck,x_target_body_center,x_target_ear_left,x_target_ear_right,x_target_lateral_left,x_target_lateral_right,x_target_nose,x_target_tail_base,x_target_tail_tip,x_target_tail_midpoint,x_target_neck,y_target_body_center,y_target_ear_left,y_target_ear_right,y_target_lateral_left,y_target_lateral_right,y_target_nose,y_target_tail_base,y_target_tail_tip,y_target_tail_midpoint,y_target_neck,dx,dy,dist
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1149.209961,1157.660034,1116.16394,1172.079956,1132.894043,1116.237061,1165.696045,330.029999,null,null,445.333008,516.973999,485.903015,476.265015,398.407013,526.552002,389.093994,367.631989,null,null,1104.375977,1123.755981,1072.952026,1128.312988,1075.534058,1110.120972,1081.697998,980.187012,1030.911011,null,304.947998,332.627014,342.309998,290.011993,308.338989,380.158997,274.985992,188.085999,234.401993,null,-44.833984,-140.38501,147.370407
1,1151.474976,1152.540039,1138.628052,1171.180054,1164.267944,1102.490967,1163.123047,330.640991,null,null,452.80899,521.947021,405.372986,482.645996,402.563995,383.053986,396.161011,367.600006,null,null,1105.51001,1124.618042,1072.762939,1134.557007,1082.232056,null,1094.462036,980.932007,1047.628052,null,315.009003,336.235992,340.915985,312.863007,302.312988,null,280.283997,194.697006,231.712006,null,-45.964966,-137.799988,145.263947
2,1151.827026,1153.832031,1132.987061,1171.368042,1156.902954,null,1165.732056,330.735992,null,null,453.035004,520.281982,405.522003,482.233002,403.054993,null,397.067993,369.046997,null,null,1103.655029,1119.79895,1068.567017,1132.973022,1075.192993,1099.938965,1094.609985,981.106995,1048.917969,null,314.234985,335.475006,340.154999,316.86499,305.850006,385.057007,281.042999,194.479004,231.863007,null,-48.171997,-138.800018,146.921707
3,1151.515991,1151.015015,1133.811035,1170.697021,1156.859009,null,1165.370972,330.743988,null,null,453.364014,520.856995,405.924011,483.083008,402.816986,null,396.713013,367.914001,null,null,1098.209961,1119.234009,1064.43103,1131.587036,1070.720947,1097.875,1093.571045,979.241028,1046.725952,null,318.411987,340.132996,346.139008,314.371002,311.549988,391.467987,280.559998,195.647003,230.904007,null,-53.30603,-134.952026,145.098526
4,1150.463989,1150.425049,1136.776001,1170.224976,1132.267944,null,1165.357056,null,null,null,454.019012,521.846985,403.516998,483.425995,453.058014,null,396.131989,null,null,null,1095.712036,1115.084961,1058.192017,1125.430054,1070.906006,1092.083008,1091.31897,973.208984,1047.359009,null,320.815002,341.51001,350.02301,306.688995,313.287994,397.033997,284.833008,188.957993,231.783997,null,-54.751953,-133.20401,144.017654
5,1150.41394,1148.480957,1135.156006,1169.813965,null,null,1164.758057,null,null,null,453.595001,523.04303,405.962006,484.686005,null,null,396.13501,null,null,null,1094.618042,1114.537964,1054.892944,1124.831055,1070.761963,1093.89502,1089.692993,980.336975,1048.54895,null,320.144989,343.378998,354.856995,306.085999,315.998993,394.795013,286.595001,193.626007,232.919006,null,-55.795898,-133.450012,144.644699
6,1150.467041,1148.758057,1136.973022,1170.001953,1124.781982,null,1165.926025,null,1158.162964,null,455.716003,523.242981,406.184998,484.928009,459.809998,null,397.696991,null,325.351013,null,1095.182983,1112.090942,1056.958008,1125.233032,1070.355957,1096.082031,1090.44397,978.117004,1048.109009,null,320.196014,342.21701,357.438995,306.28299,314.829987,395.584015,286.847992,189.291,232.798996,null,-55.284058,-135.519989,146.362549
7,1149.87902

video_frame,x_agent_body_center,x_agent_ear_left,x_agent_ear_right,x_agent_lateral_left,x_agent_lateral_right,x_agent_nose,x_agent_tail_base,x_agent_tail_tip,x_agent_tail_midpoint,x_agent_neck,y_agent_body_center,y_agent_ear_left,y_agent_ear_right,y_agent_lateral_left,y_agent_lateral_right,y_agent_nose,y_agent_tail_base,y_agent_tail_tip,y_agent_tail_midpoint,y_agent_neck,x_target_body_center,x_target_ear_left,x_target_ear_right,x_target_lateral_left,x_target_lateral_right,x_target_nose,x_target_tail_base,x_target_tail_tip,x_target_tail_midpoint,x_target_neck,y_target_body_center,y_target_ear_left,y_target_ear_right,y_target_lateral_left,y_target_lateral_right,y_target_nose,y_target_tail_base,y_target_tail_tip,y_target_tail_midpoint,y_target_neck,dx,dy,dist
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
81770,602.614014,null,null,596.330994,597.541016,605.629028,575.034973,564.231995,558.41803,null,68.438004,null,null,51.455002,89.972,106.583,77.025002,108.372002,125.962997,null,581.877014,614.080994,572.971008,614.033997,552.494995,620.247986,579.781006,null,593.757019,null,925.432007,937.112976,969.551025,909.768005,936.557983,992.491028,890.362,null,818.609985,null,-20.737,856.994019,857.244873
81771,602.771973,null,null,620.156982,597.541016,605.585022,574.906982,564.294983,558.375,null,68.296997,null,null,83.035004,89.670998,106.564003,77.223999,108.408997,126.110001,null,581.892029,615.862976,574.892029,612.877014,552.374023,620.429016,579.52301,null,593.976013,null,925.815979,937.91803,970.252014,909.60199,938.219971,992.669983,890.067017,null,817.979004,null,-20.879944,857.518982,857.773193
81772,602.896973,null,null,620.151978,597.650024,605.66803,574.893982,564.338989,558.267029,null,68.325996,null,null,83.078003,89.689003,106.547997,77.399002,108.431999,126.806,null,581.903015,616.036987,573.86499,612.97998,550.317017,619.976013,579.525024,null,593.91803,null,925.749023,938.067993,968.896973,909.343018,931.127991,993.698975,890.05603,null,818.229004,null,-20.993958,857.423035,857.679993
81773,602.89502,null,null,596.434021,597.476013,605.596008,574.801025,564.333984,558.210999,null,68.286003,null,null,51.636002,89.633003,106.607002,77.275002,108.416,126.753998,null,581.067017,615.328003,574.169983,612.857971,549.98999,620.377991,579.619019,null,593.982971,null,925.481018,937.336975,970.64801,910.213013,930.841003,993.505981,890.096985,null,817.945007,null,-21.828003,857.195007,857.472839
81774,602.71698,null,null,596.494995,597.362976,605.572998,574.674988,564.38501,558.218018,null,68.481003,null,null,51.682999,89.497002,106.589996,77.166,108.402,126.305,null,581.455017,617.435974,574.135986,613.39502,550.577026,620.429016,579.986023,null,594.02301,null,926.21698,936.580017,970.247986,910.52002,931.744995,994.489014,890.153015,null,817.642029,null,-21.261963,857.735962,857.999451
81775,602.741028,null,null,620.182007,595.492004,605.651001,574.768982,564.398987,558.060974,null,68.639999,null,null,82.999001,94.714996,106.435997,77.571999,108.408997,126.475998,null,581.336975,616.856018,575.364014,613.880981,550.901978,622.231018,579.583984,null,594.156982,null,926.731018,937.192993,972.070007,909.742981,929.947021,997.468994,890.312012,null,817.838989,null,-21.404053,858.091003,858.35791
81776,602.661011,null,null,620.239014,597.448975,null,574.502014,564.424011,557.757996,null,68.542999,null,null,82.915001,89.639999,null,77.473999,108.346001,126.721001,null,580.950012,617.349976,575.271973,614.304016,551.948975,622.036011,579.630981,null,594.158997,null,926.630981,937.377991,971.585022,910.08197,936.455017,997.242004,890.247009,null,817.869019,null,-21.710999,858.088013,858.362671
81777,602.534973,null,null,620.192017,597.559998,null,574.695984,564.442993,557.835999,null,68.638,null,null,82.982002,89.660004,null,77.628998,108.317001,126.516998,null,581.731995,617.210999,575.77301,614.906982,

In [20]:
video_id=44566106
train_meta = train_dataframe.clone()

body_parts = (
    train_meta
    .filter(pl.col("video_id") == video_id)
    ["body_parts_tracked"].item()
)
print(body_parts)

["body_center", "ear_left", "ear_right", "headpiece_bottombackleft", "headpiece_bottombackright", "headpiece_bottomfrontleft", "headpiece_bottomfrontright", "headpiece_topbackleft", "headpiece_topbackright", "headpiece_topfrontleft", "headpiece_topfrontright", "lateral_left", "lateral_right", "neck", "nose", "tail_base", "tail_midpoint", "tail_tip"]


In [21]:
def load_and_pivot_tracking(
    video_id: int,
    train_meta: pl.DataFrame,   # video_meta から該当行を引く
    config,
) -> pl.DataFrame:
    """1動画の tracking を (video_frame, mouse_id) index に pivot する"""
    # video_id から tracking path を取得
    path = video2tracking[video_id]
    df = pl.read_parquet(path)  # columns: video_frame, mouse_id, bodypart, x, y, [likelihood]
    df = df.select(["video_frame", "mouse_id", "bodypart", "x", "y"])

    # body_parts_tracked から、この動画で本当にトラッキングされている bodypart を取得
    body_parts = (
        train_meta
        .filter(pl.col("video_id") == video_id)
        ["body_parts_tracked"].item()
    )
    body_parts = json.loads(body_parts)
    use_parts = [bp for bp in config.BODY_PARTS if bp in body_parts]

    df = df.filter(pl.col("bodypart").is_in(use_parts))

    # ここで x, y の欠損処理（NaN や 0,0 を None にするなど）を入れても良い


    # pivot
    df_pivot = (
        df.pivot(
            index=["video_frame", "mouse_id"],
            columns="bodypart",
            values=["x", "y"],
        )
        .sort(["video_frame", "mouse_id"])
    )

    # ない bodypart 列を Null で追加して列そろえ
    for bp in config.BODY_PARTS:
        for prefix in ("x_", "y_"):
            col = f"{prefix}{bp}"
            if col not in df_pivot.columns:
                df_pivot = df_pivot.with_columns(
                    pl.lit(None, dtype=pl.Float32).alias(col)
                )

    # ここで補間や smoothing を入れるなら:
    # position_cols = [c for c in df_pivot.columns if c.startswith("x_") or c.startswith("y_")]
    # df_pivot = (
    #     df_pivot
    #     .groupby("mouse_id", maintain_order=True)
    #     .map_groups(lambda g: g.sort("video_frame").with_columns(
    #         [pl.col(c).interpolate().alias(c) for c in position_cols]
    #     ))
    # )

    return df_pivot

In [22]:
def rename_agent(col: str) -> str:
    if col in ("video_frame", "mouse_id"):
        return col
    if col.startswith("x_"):
        return "x_agent_" + col[2:]
    if col.startswith("y_"):
        return "y_agent_" + col[2:]
    return col

def rename_target(col: str) -> str:
    if col in ("video_frame", "mouse_id"):
        return col
    if col.startswith("x_"):
        return "x_target_" + col[2:]
    if col.startswith("y_"):
        return "y_target_" + col[2:]
    return col

def make_pair_features_single(
    df_pivot: pl.DataFrame,
    agent_id: int,
    target_id: int,
) -> pl.DataFrame:
    agent = (
        df_pivot
        .filter(pl.col("mouse_id") == agent_id)
        .rename(rename_agent)
    )
    target = (
        df_pivot
        .filter(pl.col("mouse_id") == target_id)
        .rename(rename_target)
    )
    pair = (
        agent.drop("mouse_id")
        .join(target.drop("mouse_id"), on="video_frame", how="inner")
        .sort("video_frame")
    )

    # 例として body_center の相対位置・距離を入れている
    pair = pair.with_columns(
        (pl.col("x_target_body_center") - pl.col("x_agent_body_center")).alias("dx_body_center"),
        (pl.col("y_target_body_center") - pl.col("y_agent_body_center")).alias("dy_body_center"),
    ).with_columns(
        (pl.col("dx_body_center") ** 2 + pl.col("dy_body_center") ** 2)
        .sqrt()
        .alias("dist_body_center"),
    )

    return pair  # video_frame, x_agent_*, y_agent_*, x_target_*, y_target_*, dx, dy, dist, ...

In [23]:
pair_list = (
    behavior_pairs
    .filter(pl.col("video_id") == video_id)
    .select("agent_mouse_id", "target_mouse_id")
    .unique()
)
print(pair_list)

shape: (16, 2)
┌────────────────┬─────────────────┐
│ agent_mouse_id ┆ target_mouse_id │
│ ---            ┆ ---             │
│ i8             ┆ i8              │
╞════════════════╪═════════════════╡
│ 4              ┆ 3               │
│ 2              ┆ 3               │
│ 2              ┆ 1               │
│ 1              ┆ 1               │
│ 3              ┆ 4               │
│ …              ┆ …               │
│ 3              ┆ 3               │
│ 4              ┆ 2               │
│ 2              ┆ 4               │
│ 4              ┆ 4               │
│ 1              ┆ 4               │
└────────────────┴─────────────────┘


In [24]:
# pair_list.to_dicts()

In [25]:
def build_video_pair_features(
    video_id: int,
    df_pivot: pl.DataFrame,
    behavior_pairs: pl.DataFrame,  # behavior_pairs 全体
) -> pl.DataFrame:
    # この動画で評価対象の (agent, target) ペア一覧, pl.DataFrame
    pair_list = (
        behavior_pairs
        .filter(pl.col("video_id") == video_id)
        .select("agent_mouse_id", "target_mouse_id")
        .unique()
    )

    # 各ペアごとに特徴量を作成して結合 
    pair_features_list = []
    for row in pair_list.to_dicts(): # to_dicts -> [{'agent_mouse_id': 1, 'target_mouse_id': 2}, ...]
        a = int(row["agent_mouse_id"])
        t = int(row["target_mouse_id"])
        # make_pair_features_singleを呼び出し、ペア特徴量を取得
        pair = make_pair_features_single(df_pivot, agent_id=a, target_id=t)
        pair = pair.with_columns(
            pl.lit(video_id).alias("video_id"), # video_id 列を追加
            pl.lit(a).alias("agent_mouse_id"),
            pl.lit(t).alias("target_mouse_id"),
        )
        pair_features_list.append(pair)
    if not pair_features_list:
        return pl.DataFrame()  # ラベルだけあって tracking が無い、などは基本ないはず
    # concat
    pair_features = pl.concat(pair_features_list)

    # 最後に INDEX_COLS を先頭にそろえておく
    pair_features = pair_features.select(
        [
            "video_id",
            "agent_mouse_id",
            "target_mouse_id",
            "video_frame",
            *[c for c in pair_features.columns
              if c not in ("video_id", "agent_mouse_id", "target_mouse_id", "video_frame")],
        ]
    )

    # 欠損マスク列を足してから 0 埋めするのもここでやると楽
    float_cols = [c for c in pair_features.columns if pair_features.schema[c] == pl.Float32 or pair_features.schema[c] == pl.Float64]
    mask_cols = []
    for c in float_cols:
        m = f"{c}_is_null"
        pair_features = pair_features.with_columns(
            pl.col(c).is_null().cast(pl.Int8).alias(m)
        )
        mask_cols.append(m)
    pair_features = pair_features.with_columns(
        [pl.col(c).fill_null(0.0) for c in float_cols]
    )

    return pair_features

In [26]:
build_video_pair_features(
    video_id=44566106,
    df_pivot=df_pivot,
    behavior_pairs=behavior_pairs,  # behavior_pairs 全体
)

video_id,agent_mouse_id,target_mouse_id,video_frame,x_agent_body_center,x_agent_ear_left,x_agent_ear_right,x_agent_lateral_left,x_agent_lateral_right,x_agent_nose,x_agent_tail_base,x_agent_tail_tip,x_agent_tail_midpoint,x_agent_neck,y_agent_body_center,y_agent_ear_left,y_agent_ear_right,y_agent_lateral_left,y_agent_lateral_right,y_agent_nose,y_agent_tail_base,y_agent_tail_tip,y_agent_tail_midpoint,y_agent_neck,x_target_body_center,x_target_ear_left,x_target_ear_right,x_target_lateral_left,x_target_lateral_right,x_target_nose,x_target_tail_base,x_target_tail_tip,x_target_tail_midpoint,x_target_neck,y_target_body_center,y_target_ear_left,y_target_ear_right,…,x_agent_tail_base_is_null,x_agent_tail_tip_is_null,x_agent_tail_midpoint_is_null,x_agent_neck_is_null,y_agent_body_center_is_null,y_agent_ear_left_is_null,y_agent_ear_right_is_null,y_agent_lateral_left_is_null,y_agent_lateral_right_is_null,y_agent_nose_is_null,y_agent_tail_base_is_null,y_agent_tail_tip_is_null,y_agent_tail_midpoint_is_null,y_agent_neck_is_null,x_target_body_center_is_null,x_target_ear_left_is_null,x_target_ear_right_is_null,x_target_lateral_left_is_null,x_target_lateral_right_is_null,x_target_nose_is_null,x_target_tail_base_is_null,x_target_tail_tip_is_null,x_target_tail_midpoint_is_null,x_target_neck_is_null,y_target_body_center_is_null,y_target_ear_left_is_null,y_target_ear_right_is_null,y_target_lateral_left_is_null,y_target_lateral_right_is_null,y_target_nose_is_null,y_target_tail_base_is_null,y_target_tail_tip_is_null,y_target_tail_midpoint_is_null,y_target_neck_is_null,dx_body_center_is_null,dy_body_center_is_null,dist_body_center_is_null
i32,i32,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
44566106,4,2,0,264.707001,245.007996,237.647995,284.531006,261.359009,188.632996,295.811005,0.0,342.378998,0.0,529.129028,561.588013,517.10199,554.869019,495.196014,538.15802,503.34201,0.0,447.311005,0.0,1104.375977,1123.755981,1072.952026,1128.312988,1075.534058,1110.120972,1081.697998,980.187012,1030.911011,0.0,304.947998,332.627014,342.309998,…,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
44566106,4,2,1,269.755005,246.979004,241.117996,291.902008,264.782013,196.436996,303.700989,0.0,346.834991,0.0,527.312988,562.159973,521.320007,546.619995,492.790009,527.757019,502.001007,0.0,446.829987,0.0,1105.51001,1124.618042,1072.762939,1134.557007,1082.232056,0.0,1094.462036,980.932007,1047.628052,0.0,315.009003,336.235992,340.915985,…,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
44566106,4,2,2,273.295013,248.285995,246.292007,287.394989,271.584991,196.388,304.661987,0.0,348.813995,0.0,527.046997,558.210022,520.366028,548.804993,495.036987,517.812012,502.984985,0.0,439.544006,0.0,1103.655029,1119.79895,1068.567017,1132.973022,1075.192993,1099.938965,1094.609985,981.106995,1048.917969,0.0,314.234985,335.475006,340.154999,…,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
44566106,4,2,3,271.576996,243.936005,251.455002,290.696991,274.554993,201.748001,309.76001,0.0,350.214996,0.0,523.734985,548.590027,515.499023,551.187988,492.057007,511.752014,500.343994,0.0,439.851013,0.0,1098.209961,1119.234009,1064.43103,1131.587036,1070.720947,1097.875,1093.571045,979.241028,1046.725952,0.0,318.411987,340.132996,346.139008,…,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
44566106,4,2,4,277.273987,242.834,256.955994,271.734985,275.717987,205.595993,314.415985,330.585999,1160.564941,0.0,523.343994,545.278015,503.329987,557.034973,490.911011,503.269989,499.82901,367.432007,330.730988,0.0,1095.712036,1115.084961,1058.192017,1125.430054,1070.906006,1092.083008,1091.31897,973.208984,1047.359009,0.0,320.815002,341.51001,350.02301,…,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,

In [27]:
def build_frame_labels_for_video(
    video_id: int,
    pair_features_video: pl.DataFrame,
    events: pl.DataFrame,           # annot のうち該当 video_id だけ
    behavior_pairs: pl.DataFrame,   # behavior_pairs のうち該当 video_id だけ
) -> pl.DataFrame:
    df = pair_features_video.select(
        "video_id", "agent_mouse_id", "target_mouse_id", "video_frame"
    )

    # この動画で評価される (agent, target, action) 一覧
    active = (
        behavior_pairs
        .filter(pl.col("video_id") == video_id)
        .select("agent_mouse_id", "target_mouse_id", "action")
        .unique()
    )

    # action ごとに列を作る（すべて 0 で初期化）
    actions = active["action"].unique().to_list()
    for act in actions:
        df = df.with_columns(pl.lit(0, dtype=pl.Int8).alias(f"label_{act}"))

    # イベントごとに対応するフレームに 1 を立てる
    # （実装は pandas に落として for-loop でやる方が書きやすいかも）
    for row in events.filter(pl.col("video_id") == video_id).to_dicts():
        a = int(row["agent_mouse_id"])
        t = int(row["target_mouse_id"])
        act = row["action"]
        s = int(row["start_frame"])
        e = int(row["stop_frame"])

        colname = f"label_{act}"

        df = df.with_columns(
            pl.when(
                (pl.col("agent_mouse_id") == a)
                & (pl.col("target_mouse_id") == t)
                & (pl.col("video_frame") >= s)
                & (pl.col("video_frame") < e)
            )
            .then(1)
            .otherwise(pl.col(colname))
            .alias(colname)
        )

    return df

# Annotation data

In [28]:
# =============================
# gather annotation paths
# =============================
annotation_paths = glob_walk(config.TRAIN_ANNOTATION_DIR, "**/*.parquet")
video2annotation = build_video_path_dict(annotation_paths)
print(f"length of annotation paths: {len(annotation_paths)}")
print(annotation_paths[:3])

length of annotation paths: 847
[PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_annotation/AdaptableSnail/1212811043.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_annotation/AdaptableSnail/1260392287.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_annotation/AdaptableSnail/1351098077.parquet')]


In [29]:
df_annotation_sample = pl.read_parquet(annotation_paths[0])
sep("df_annotation_sample"); show_df(df_annotation_sample)

df_annotation_sample
(370, 5)


agent_id,target_id,action,start_frame,stop_frame
i8,i8,str,i32,i32
1,3,"""chase""",2,54
1,3,"""chase""",128,234
3,2,"""avoid""",324,342


In [ ]:
def load_all_annotations(
    annotation_paths: list[Path],
    train_dataframe: pl.DataFrame,
) -> pl.DataFrame:
    meta_small = train_dataframe.select("video_id", "lab_id", "behaviors_labeled")

    dfs = []
    for p in tqdm(annotation_paths, desc="Loading annotations"):
        df = pl.read_parquet(p)
        # video_id が parquet に無ければファイル名から補完
        if "video_id" not in df.columns:
            vid = int(p.stem)
            df = df.with_columns(pl.lit(vid).alias("video_id"))

        df = df.select(
            "video_id",
            "agent_id",
            "target_id",
            "action",
            "start_frame",
            "stop_frame",
        )

        dfs.append(df)

    annot = pl.concat(dfs, how="vertical_relaxed")

    # agent_id, target_id は既に 1..4 の int で入っている想定
    annot = annot.with_columns(
        pl.col("agent_id").cast(pl.Int8).alias("agent_mouse_id"),
        pl.col("target_id").cast(pl.Int8).alias("target_mouse_id"),
    ).drop(["agent_id", "target_id"])

    # lab_id と behaviors_labeled を付けて metric と同じ形に近づける
    annot = (
        annot.join(meta_small, on="video_id", how="left")
        .rename({"behaviors_labeled": "behaviors_labeled_json"})
    )

    # 念のため start <= stop をチェック
    assert (annot["start_frame"] <= annot["stop_frame"]).all()

    return annot  # これが「イベントラベル」

In [31]:
all_annotations = load_all_annotations(
    annotation_paths=annotation_paths,
    train_dataframe=train_dataframe,
)
sep("all_annotations"); show_df(all_annotations, 3, True)

Loading annotations: 100%|██████████| 847/847 [00:10<00:00, 77.48it/s]


all_annotations
(83620, 8)


video_id,action,start_frame,stop_frame,agent_mouse_id,target_mouse_id,lab_id,behaviors_labeled_json
i32,str,i32,i32,i8,i8,str,str
1212811043,"""chase""",2,54,1,3,"""AdaptableSnail""","""[""mouse1,mouse2,approach"", ""mo…"
1212811043,"""chase""",128,234,1,3,"""AdaptableSnail""","""[""mouse1,mouse2,approach"", ""mo…"
1212811043,"""avoid""",324,342,3,2,"""AdaptableSnail""","""[""mouse1,mouse2,approach"", ""mo…"


video_id,action,start_frame,stop_frame,agent_mouse_id,target_mouse_id,lab_id,behaviors_labeled_json
i32,str,i32,i32,i8,i8,str,str
86824777,"""sniffgenital""",14139,14156,1,2,"""UppityFerret""","""[""mouse1,mouse2,intromit"", ""mo…"
86824777,"""sniffgenital""",14170,14174,1,2,"""UppityFerret""","""[""mouse1,mouse2,intromit"", ""mo…"
86824777,"""sniffgenital""",14316,14331,2,1,"""UppityFerret""","""[""mouse1,mouse2,intromit"", ""mo…"


In [38]:
import json

def build_video_meta_and_behavior_pairs(train_df: pl.DataFrame, config):
    # ラベル付き動画だけ
    train_labeled = (
        train_df
        .filter(pl.col("behaviors_labeled").is_not_null())
        .select(
            "lab_id",
            "video_id",
            "body_parts_tracked",
            "behaviors_labeled",
            "frames_per_second",
            "video_duration_sec",
        )
    )

    # 動画メタ情報 (body_parts_tracked / behaviors_labeled を list 化)
    video_meta = (
        train_labeled
        .with_columns([
            pl.col("body_parts_tracked")
              .map_elements(json.loads, return_dtype=pl.List(pl.Utf8))
              .alias("body_parts_tracked_list"),
            pl.col("behaviors_labeled")
              .map_elements(json.loads, return_dtype=pl.List(pl.Utf8))
              .alias("behaviors_labeled_list"),
        ])
    )

    # ↓ここは前に直した behavior_pairs の処理（ざっくり）
    behavior_pairs_raw = (
        video_meta
        .select(
            "lab_id",
            "video_id",
            "behaviors_labeled_list",
        )
        .explode("behaviors_labeled_list")
        .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
        .with_columns(
            pl.col("behaviors_labeled_element")
              .str.replace_all("'", "")
              .str.replace_all('"', "")
              .alias("behaviors_clean")
        )
        .with_columns(
            pl.col("behaviors_clean").str.split(",").alias("parts")
        )
        .select(
            "lab_id",
            "video_id",
            pl.col("parts").list.get(0).alias("agent_str"),
            pl.col("parts").list.get(1).alias("target_str"),
            pl.col("parts").list.get(2).alias("action"),
        )
    )

    def parse_mouse_id(s):
        if s is None:
            return None
        s = str(s).strip().replace("'", "").replace('"', "")
        if s.lower() == "self":
            return None
        s = s.replace("mouse", "")
        digits = "".join(ch for ch in s if ch.isdigit())
        return int(digits) if digits else None

    behavior_pairs = (
        behavior_pairs_raw
        .with_columns([
            pl.col("agent_str")
              .map_elements(parse_mouse_id, return_dtype=pl.Int8)
              .alias("agent_mouse_id"),
            pl.col("target_str")
              .map_elements(parse_mouse_id, return_dtype=pl.Int8)
              .alias("target_mouse_raw_id"),
        ])
        .with_columns(
            pl.when(pl.col("target_mouse_raw_id").is_null())
              .then(pl.col("agent_mouse_id"))
              .otherwise(pl.col("target_mouse_raw_id"))
              .alias("target_mouse_id")
        )
        .drop("target_mouse_raw_id")
        .with_columns(
            pl.when(pl.col("action").is_in(config.SELF_BEHAVIORS))
              .then(pl.lit("self"))
              .otherwise(pl.lit("pair"))
              .alias("behavior_type")
        )
        .select(
            "lab_id",
            "video_id",
            "agent_mouse_id",
            "target_mouse_id",
            "action",
            "behavior_type",
        )
        .unique()
    )

    return video_meta, behavior_pairs

In [39]:
def main_preprocess(config):
    # 1. メタデータ読込
    train_df = pl.read_csv(config.INPUT_DIR / "train.csv")
    test_df  = pl.read_csv(config.INPUT_DIR / "test.csv")

    # 2. video_meta と behavior_pairs を作る
    video_meta, behavior_pairs = build_video_meta_and_behavior_pairs(train_df, config)

    # 3. annotation 読み込み
    annotation_paths = glob_walk(config.TRAIN_ANNOTATION_DIR, "**/*.parquet")
    events = load_all_annotations(annotation_paths, train_df)  # ←ここは前に書いた関数

    # 4. ラベル付き動画だけループ
    labeled_video_ids = behavior_pairs["video_id"].unique().to_list()

    all_pair_feats = []
    all_frame_labels = []

    for vid in tqdm(labeled_video_ids):
        # tracking を pivot
        df_pivot = load_and_pivot_tracking(vid, video_meta, config)  # train_df じゃなく video_meta 渡す

        # ペア特徴
        pair_feats_vid = build_video_pair_features(vid, df_pivot, behavior_pairs)

        # ラベルイベント（この動画だけ）
        events_vid = events.filter(pl.col("video_id") == vid)

        # フレームラベル
        frame_labels_vid = build_frame_labels_for_video(
            vid,
            pair_feats_vid.select(config.INDEX_COLS),
            events_vid,
            behavior_pairs,
        )

        all_pair_feats.append(pair_feats_vid)
        all_frame_labels.append(frame_labels_vid)

    pair_features = pl.concat(all_pair_feats)
    frame_labels  = pl.concat(all_frame_labels)

    pair_features.write_parquet("train_pair_features.parquet")
    frame_labels.write_parquet("train_frame_labels.parquet")

In [ ]:
main_preprocess(config)

  0%|          | 0/848 [00:00<?, ?it/s]/tmp/ipykernel_1958556/3753844375.py:28: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  df.pivot(
 33%|███▎      | 277/848 [01:46<04:23,  2.17it/s]